https://grants.corp.google.com/#/grants?request=20h%2Fchr-ards-fitbit-prod-research-deid-eng-team:r&reason=%22b%2F285178698%22

Colab Kernel: Fitbit Prod Research Colab and please follow the steps:

- Use the Fitbit prod kernel;
- Restart the session;
- Add import tensorflow_datasets as tfds to the top; -
- Run ad_hoc import.


In [ ]:
from colabtools import adhoc_import
from colabtools import f1

from google3.pyglib import gfile
import pandas as pd

In [ ]:
DATA_KEY_TYPE = 'TIER2_SITE_DATA'
DATABASE_PATH = '/span/nonprod/consumer-health-research:fitbit-prod-research'


data_key_type_str = f"'{DATA_KEY_TYPE}'"
user_ids_df = f1.Execute(f"""
  SELECT DataKey.session_id
  FROM {DATABASE_PATH}.DataStorage
  WHERE
    StorageKey = 'momentary_stress_algorithm'
    AND DataKey.type = {data_key_type_str}
""")

In [ ]:
user_ids_df.shape

In [ ]:
WRITE_NEW_FILES = False
fp = '/namespace/fitbit-medical-sandboxes/jg/partner/encrypted/chr-ards-fitbit-prod-research/deid/exp/dmcduff/ttl=52w/lsm_v2/datasets/raw/sessions.csv'
if WRITE_NEW_FILES:
  with gfile.Open(fp, 'w') as f:
    user_ids_df.to_csv(f, index=False)
else:
  with gfile.Open(fp, 'r') as f:
    user_ids_df = pd.read_csv(f)
ids = user_ids_df['session_id']

In [ ]:
ids.shape

# Module Test

In [ ]:
with adhoc_import.Google3CitcClient(
    'lsm_v2_data_gen_fix_end_2024',
    username='xliucs',
    behavior='fallback', # Run this after "preferred".
    # behavior='preferred', # Run this first then comment out the next line.
    build_targets=[
        '//medical/waveforms/modelling/lsm/datasets/lsm:download_and_prepare'
    ],
):
  from google3.medical.waveforms.modelling.lsm.datasets.lsm import sensors
  sensors = adhoc_import.Reload(sensors, reset_flags=True)


In [ ]:
iter = sensors.window(ids[:2], '10080min', 's', 36)
key, data = next(iter)

In [ ]:
data['input_signal'].shape

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(data['input_signal'].T, aspect='auto')